## Cramer-Rao bounds for (2D) Gaussian fitting.

This notebook explains how to calculate the Cramer-Rao bounds for 2D Gaussian fitting. Basically this is the best resolution that you can a expect from a MLE Gaussian fitter like `3D-DAOSTORM` or `sCMOS`. In practice your resolution will often be worse due to issues like imperfect correction for stage drift, overlapping localizations, non-uniform background, etc.. It is also pretty common to find that your actual resolution is limited by labeling density. For example many antibodies that are good enough for conventional imaging will not reach the labeling density necessary to generate good SMLM images.

References:
* [Ober et al, Biophysical Journal, 2004](https://doi.org/10.1016/S0006-3495%2804%2974193-4).
* [Mortensen et al, Nature Methods, 2010](http://dx.doi.org/10.1038/nmeth.1447).

### Configuring the directory
Create an empty directory somewhere on your computer and tell Python to go to that directory.

In [ ]:
import os
os.chdir("/home/hbabcock/Data/storm_analysis/jy_testing/")
print(os.getcwd())

Generate the XML file for `3D-DAOSTORM` analysis.

In [ ]:
import numpy
import storm_analysis.jupyter_examples.dao3d_crao as dao3d_crao

dao3d_crao.createParametersFile()

### Calculating the Cramer-Rao bounds

We're going to use `scipy.integrate.quad` to numerically integrate equation 5 in *Mortensen et al*.

\begin{equation*}
Variance(\mu_x) = \frac{\sigma^2_a}{N}\left(1+\int_{0}^{1}\frac{\ln{t}}{1+\frac{Na^2t}{2\pi\sigma^2_ab^2}}dt\right)^{-1}
\end{equation*}

In this equation:  
$\mu_x$ is the estimate of the localization position.  
$N$ is the integrated PSF intensity in $e^-$ (photo-electrons).  
$b^2$ is the per pixel background in $e^-$.  
$a$ is the pixel size in nanometers.  
$\sigma_a$ is the PSF $\sigma$ in nanometers.  

This is available as a function in the `storm_analysis.sa_utilities.mortensen` module. The function can calculate the expected $\sigma$ for an sCMOS or an EMCCD camera. In this example we are simulating an idealized sCMOS camera which has no readout noise.

In [ ]:
import storm_analysis.sa_utilities.mortensen as mortensen

dao3d_crao.signal = 2000
dao3d_crao.bg = 50

# The parameters that we'll used to generate the simulated data.
print("N:", dao3d_crao.signal)
print("b2:", dao3d_crao.bg)
print("pixel size:", dao3d_crao.pixel_size)
print("PSF sigma:", 1.5*dao3d_crao.pixel_size) # 1.5 pixels is the default PSF sigma used in these simulations.
print("")

cr_sigma = mortensen.cramerRaoBound(dao3d_crao.signal, 
                                    dao3d_crao.bg, 
                                    dao3d_crao.pixel_size, 
                                    dao3d_crao.pixel_size * 1.5)
print("Cramer-Rao fit sigma: {0:.3f}nm".format(cr_sigma))

### Generate simulated data

In [ ]:
# Create localizations on a grid.
dao3d_crao.createLocalizations()

# Generate a 40 frame movie. We'll need a few thousand localizations to get good statistics.
dao3d_crao.createMovie(40)


### Analyze the simulated data with 3D-DAOSTORM

In this example we are going to use the ability of `3D-DAOSTORM` to fit Gaussians starting at user defined positions. This is often convenient for simulated data where you just want to know how well fitting can be done without worrying about also finding the localizations.

Note that even though the fitter is given the exactly correct position for each localization it will not return this position due to the noise in the image.

In [ ]:
import storm_analysis.sa_library.parameters as params
daop = params.ParametersDAO().initFromFile("dao3d_crao.xml")

print(daop.helpAttr("peak_locations"))

print("")
print("peak_locations:", daop.getAttr("peak_locations"))

In [ ]:
import storm_analysis.daostorm_3d.mufit_analysis as mfit

# Remove stale results, if any.
if os.path.exists("testing.hdf5"):
    os.remove("testing.hdf5")
    
# (Re)run the analysis.
mfit.analyze("test.tif", "testing.hdf5", "dao3d_crao.xml")

### Compare simulation error to theoritical error

In [ ]:
import storm_analysis.sa_utilities.finding_fitting_error as ffe

[dx, dy, dz] = ffe.findingFittingErrorHDF5File("test_ref.hdf5", "testing.hdf5")

print("Theoritical fitting error (sigma) {0:.3f}nm".format(cr_sigma))
print("Measured fitting error (sigma) {0:.3f}nm".format(0.5*(numpy.std(dx) + numpy.std(dy))))
